In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [20]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler,PowerTransformer
from sklearn.decomposition import PCA
from sklearn.mixture import BayesianGaussianMixture,GaussianMixture
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from sklearn import metrics
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [22]:
df=pd.read_csv("/kaggle/input/asasas/GiaSMP.csv" , encoding = 'ISO-8859-1')


In [38]:
df1 = pd.concat([df["Ngày"], df["6"]] , axis = 1)
df1.head(10)

,Ngày,6
0,1/1/2021,964.4
1,2/1/2021,1019.7
2,3/1/2021,988.4
3,4/1/2021,1002.0
4,5/1/2021,1061.5
5,6/1/2021,1061.5
6,7/1/2021,1061.5
7,8/1/2021,1061.5
8,9/1/2021,1064.1
9,10/1/2021,966.7


In [42]:
feats = ['6']

In [43]:
df1.head()

,Ngày,6
0,1/1/2021,964.4
1,2/1/2021,1019.7
2,3/1/2021,988.4
3,4/1/2021,1002.0
4,5/1/2021,1061.5


In [44]:
df1.info

<bound method DataFrame.info of            Ngày       6
0      1/1/2021   964.4
1      2/1/2021  1019.7
2      3/1/2021   988.4
3      4/1/2021  1002.0
4      5/1/2021  1061.5
..          ...     ...
360  12/27/2021  1002.0
361  12/28/2021  1002.0
362  12/29/2021  1061.5
363  12/30/2021  1022.6
364  12/31/2021  1022.6

[365 rows x 2 columns]>

In [45]:
from sklearn.preprocessing import PowerTransformer

# Chuyển đổi cột '11' thành mảng 2D
X = df['11'].values.reshape(-1, 1)

# Áp dụng PowerTransformer
transformer = PowerTransformer()
X_transformed = transformer.fit_transform(X)


In [46]:
BGM = BayesianGaussianMixture(n_components=7,covariance_type='full',random_state=1,n_init=15)
# fit model and predict clusters
preds = BGM.fit_predict(X)

#Adding the Clusters feature to the orignal dataframe.
df["Clusters"]= preds

In [47]:
pp=BGM.predict_proba(X)# Calcualting the probabilities of each prediction
df_new=pd.DataFrame(X,columns=feats) 
df_new[[f'predict_proba_{i}' for i in range(7)]]=pp # creating new dataframe columns of probabilites 
df_new['preds']=preds
df_new['predict_proba']=np.max(pp,axis=1)
df_new['predict']=np.argmax(pp,axis=1)
    
train_index=np.array([])
for n in range(7):
    n_inx=df_new[(df_new.preds==n) & (df_new.predict_proba > 0.68)].index
    train_index = np.concatenate((train_index, n_inx))

In [48]:
from sklearn.model_selection import StratifiedKFold
X_new=df_new.loc[train_index][feats]
y=df_new.loc[train_index]['preds']

params_lgb = {'learning_rate': 0.06,'objective': 'multiclass','boosting': 'gbdt','n_jobs': -1,'verbosity': -1, 'num_classes':7} 

model_list=[]

gkf = StratifiedKFold(11)
for fold, (train_idx, valid_idx) in enumerate(gkf.split(X_new,y)):   

    tr_dataset = lgb.Dataset(X_new.iloc[train_idx],y.iloc[train_idx],feature_name = feats)
    vl_dataset = lgb.Dataset(X_new.iloc[valid_idx],y.iloc[valid_idx],feature_name = feats)
    
    model = lgb.train(params = params_lgb, 
                train_set = tr_dataset, 
                valid_sets =  vl_dataset, 
                num_boost_round = 5000, 
                callbacks=[ lgb.early_stopping(stopping_rounds=300, verbose=False), lgb.log_evaluation(period=200)])  
    
    model_list.append(model) 


[200]	valid_0's multi_logloss: 0.000151038
[400]	valid_0's multi_logloss: 0.000151038
[200]	valid_0's multi_logloss: 0.000151576
[400]	valid_0's multi_logloss: 0.000151576
[200]	valid_0's multi_logloss: 0.00015433
[400]	valid_0's multi_logloss: 0.00015433
[200]	valid_0's multi_logloss: 0.000155486
[400]	valid_0's multi_logloss: 0.000155486
[200]	valid_0's multi_logloss: 0.000153559
[400]	valid_0's multi_logloss: 0.000153559
[200]	valid_0's multi_logloss: 0.000155711
[400]	valid_0's multi_logloss: 0.000155711
[200]	valid_0's multi_logloss: 8.58365e-06
[400]	valid_0's multi_logloss: 8.48634e-06
[600]	valid_0's multi_logloss: 8.42158e-06
[800]	valid_0's multi_logloss: 8.37546e-06
[1000]	valid_0's multi_logloss: 8.34101e-06
[1200]	valid_0's multi_logloss: 8.3143e-06
[1400]	valid_0's multi_logloss: 8.29302e-06
[1600]	valid_0's multi_logloss: 8.27566e-06
[1800]	valid_0's multi_logloss: 8.26123e-06
[2000]	valid_0's multi_logloss: 8.24906e-06
[2200]	valid_0's multi_logloss: 8.23866e-06
[2400]	

In [49]:
lgb_preds=0
for model in model_list:
    lgb_preds+=model.predict(df_new[feats])

In [50]:
labels=np.argmax(lgb_preds,axis=1)